# Ideas
- what to show at parcel vs taz level
- taz level comparisons for different modeling runs?

In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd
import glob
import imageio

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

In [2]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "scratch.gdb", 'results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

In [4]:
parcels = r'.\\Inputs\\parcels.gdb\\parcels_2019_20220429'
regions = r".\Inputs\REMM_Reporting_Areas.shp"
taz832 = r".\Inputs\TAZ_832_ID_Only.shp"

In [5]:
# spatial join to get the subarea
target_features = parcels
join_features = regions
output_features = os.path.join(gdb, "_01_parcels_regions")

fieldmappings = arcpy.FieldMappings()
fieldmappings.addTable(target_features)
fieldmappings.addTable(join_features)

# attribute to summarize
fieldindex = fieldmappings.findFieldMapIndex('NewSA')
fieldmap = fieldmappings.getFieldMap(fieldindex)
fieldmap.mergeRule = 'First'
fieldmappings.replaceFieldMap(fieldindex, fieldmap)

# run the spatial join
sj = arcpy.SpatialJoin_analysis(target_features, join_features, output_features,'JOIN_ONE_TO_ONE', "KEEP_ALL", 
                           fieldmappings, "HAVE_THEIR_CENTER_IN")

In [7]:
parcels_sdf = pd.DataFrame.spatial.from_featureclass(sj[0])
taz832_sdf = pd.DataFrame.spatial.from_featureclass(taz832)

In [8]:
taz832_sdf.columns

Index(['FID', 'TAZID', 'SORT', 'CO_TAZID', 'SUBAREAID', 'EXTERNAL', 'ACRES',
       'DEVACRES', 'X', 'Y', 'ADJ_XY', 'CBD', 'TERMTIME', 'PRKCSTPERM',
       'PRKCSTTEMP', 'WALK100', 'ECOEDPASS', 'FREEFARE', 'MPO', 'MPO_MSID',
       'MODEL_SPC', 'MPO_MSID2', 'MODEL_SPC2', 'COUNTY', 'CO_FIPS', 'CO_NAME',
       'CITY', 'CITY_FIPS', 'CITY_NAME', 'DISTSUPER', 'DSUP_NAME', 'DISTLRG',
       'DLRG_NAME', 'DISTMED', 'DMED_NAME', 'DISTSML', 'DSML_NAME', 'AIRSAGE',
       'WF_AIRSAGE', 'SLC', 'NAAPM25', 'SHAPE'],
      dtype='object')

In [9]:
parcels_sdf.columns

Index(['OBJECTID', 'Join_Count', 'TARGET_FID', 'parcel_id_REMM', 'CO_NAME',
       'TAZID_832', 'NewSA', 'SHAPE'],
      dtype='object')

In [10]:
# dua = parcels_sdf[['parcel_id_REMM','SHAPE']].merge(base[['parcel_id','max_dua']], left_on='parcel_id_REMM',right_on='parcel_id', how='left')
# dua.spatial.to_featureclass(location=os.path.join(gdb, "_02_base_dua"),sanitize_columns=False)

In [11]:
# dev = parcels_sdf[['parcel_id_REMM','SHAPE']].merge(base[['parcel_id','was_developed']], left_on='parcel_id_REMM',right_on='parcel_id', how='left')
# dev.spatial.to_featureclass(location=os.path.join(gdb, "_02_base_dua"),sanitize_columns=False)

In [12]:
data_folder = r".\Progression_Metrics"
csvs = glob.glob(os.path.join(data_folder,'*_parcel_progression_metrics.csv'))

In [13]:
# use any keyword to filter lists
years_to_process = ['2019','2025','2035','2045','2050']
csvs2 = [csv for csv in csvs if any(year in csv for year in years_to_process)]
csvs2

# for csv in csvs:
#     if any(year in csv for year in years_to_process):
#         print(csv)



['.\\Progression_Metrics\\run_315_year_2019_parcel_progression_metrics.csv',
 '.\\Progression_Metrics\\run_315_year_2025_parcel_progression_metrics.csv',
 '.\\Progression_Metrics\\run_315_year_2035_parcel_progression_metrics.csv',
 '.\\Progression_Metrics\\run_315_year_2045_parcel_progression_metrics.csv',
 '.\\Progression_Metrics\\run_315_year_2050_parcel_progression_metrics.csv']

In [14]:
# csvs

In [15]:
# outfiles = []
# for csv in csvs2:
#     year = os.path.basename(csv)[13:17]
#     df = pd.read_csv(csv)
#     dua = parcels_sdf[['parcel_id_REMM','SHAPE']].merge(df[['parcel_id','max_dua']], left_on='parcel_id_REMM',right_on='parcel_id', how='left')
#     del dua['parcel_id']
#     print('exporting {}'.format(os.path.join(gdb, "dua_{}".format(year))))
#     outfiles.append(os.path.join(gdb, "dua_{}".format(year)))
#     dua.spatial.to_featureclass(location=os.path.join(gdb, "dua_{}".format(year)),sanitize_columns=False)

In [16]:
outfiles = []
for csv in csvs2:
    year = os.path.basename(csv)[13:17]
    df = pd.read_csv(csv)
    parcels_dev = parcels_sdf[['parcel_id_REMM', 'TAZID_832']].merge(df[['parcel_id','was_developed']], left_on='parcel_id_REMM',right_on='parcel_id', how='left')
    taz_dev = parcels_dev.groupby('TAZID_832')[['was_developed']].sum().reset_index()
    taz_dev.columns = ['TAZID_832', 'was_developed']
    taz_join = taz832_sdf.merge(taz_dev, left_on='TAZID', right_on='TAZID_832', how='left')


    print('exporting {}'.format(os.path.join(gdb, "dev_{}".format(year))))
    outfiles.append(os.path.join(gdb, "dev_{}".format(year)))
    taz_join.spatial.to_featureclass(location=os.path.join(gdb, "dev_{}".format(year)),sanitize_columns=False)

exporting .\Outputs\scratch.gdb\dev_2019
exporting .\Outputs\scratch.gdb\dev_2025
exporting .\Outputs\scratch.gdb\dev_2035
exporting .\Outputs\scratch.gdb\dev_2045
exporting .\Outputs\scratch.gdb\dev_2050


In [17]:
# outfiles = [r'.\Outputs\scratch.gdb\dua_2019', r'.\Outputs\scratch.gdb\dua_2020', r'.\Outputs\scratch.gdb\dua_2021', r'.\Outputs\scratch.gdb\dua_2022']

In [18]:
outfiles

['.\\Outputs\\scratch.gdb\\dev_2019',
 '.\\Outputs\\scratch.gdb\\dev_2025',
 '.\\Outputs\\scratch.gdb\\dev_2035',
 '.\\Outputs\\scratch.gdb\\dev_2045',
 '.\\Outputs\\scratch.gdb\\dev_2050']

In [19]:
# project = arcpy.mp.ArcGISProject(r".\Analyze_REMM_Results.aprx")

# for f in outfiles:
#     year = os.path.basename(f)[4:8]

#     dua_lyr = arcpy.MakeFeatureLayer_management(f, "dua_lyr")
#     dua_lyrx= arcpy.SaveToLayerFile_management(dua_lyr, os.path.join(outputs[0],"dua_{}.lyrx".format(year)), "RELATIVE")
    
#     m = project.listMaps("Data")[0]
#     old_dua_lyr = m.listLayers('*dua*')[0]

#     project.updateConnectionProperties(old_dua_lyr, dua_lyrx)

    
#     # lyrFile = arcpy.mp.LayerFile(dua_lyrx)


#     layout = project.listLayouts("Test")[0]
#     if layout.mapSeries.enabled:
#         for pageNum in range(1, layout.mapSeries.pageCount + 1):
#             layout.mapSeries.currentPageNumber = pageNum
#             # layout.exportToJPEG(os.path.join(r'.\Outputs', f"{layout.mapSeries.pageRow.NewSA}.jpg"),300)
#             layout.exportToJPEG(os.path.join(r'.\Outputs', "{}_MaxDUA_{}.jpg".format(layout.mapSeries.pageRow.NewSA, year)),300)


In [ ]:
project = arcpy.mp.ArcGISProject(r".\Analyze_REMM_Results.aprx")

jpgs = []
for f in outfiles:
    year = os.path.basename(f)[4:8]

    dev_lyr = arcpy.MakeFeatureLayer_management(f, "dev_lyr")
    # dev_lyrx= arcpy.SaveToLayerFile_management(dev_lyr, os.path.join(outputs[0],"dev_{}.lyrx".format(year)), "RELATIVE")
    
    m = project.listMaps("Data")[0]
    old_dev_lyr = m.listLayers('*dev*')[0]

    old_dev_lyr.updateConnectionProperties(old_dev_lyr.connectionProperties, dev_lyr[0].connectionProperties)

    
    # lyrFile = arcpy.mp.LayerFile(dua_lyrx)


    layout = project.listLayouts("Test")[0]
    if layout.mapSeries.enabled:
        for pageNum in range(1, layout.mapSeries.pageCount + 1):
            layout.mapSeries.currentPageNumber = pageNum

            yearText = layout.listElements("TEXT_ELEMENT", "year")[0]
            yearText.text = year

            jpg = os.path.join(r'.\Outputs', "{}_Dev_{}.jpg".format(layout.mapSeries.pageRow.NewSA, year))
            jpgs.append(jpg)
            layout.exportToJPEG(jpg,300)


In [ ]:
images = []
jpgs.sort()
for jpg in jpgs:
    images.append(imageio.imread(jpg))
    

with imageio.get_writer(r'.\Outputs\Parcels_Developed_Full_Region.gif', mode='I', duration = 1) as writer:
    for jpg in jpgs:
        image = imageio.imread(jpg)
        writer.append_data(image)

In [ ]:
del project